In [0]:
use catalog pricing_analytics;
use schema silver;

create or replace table silver.reporting_dim_market_stage_1 AS 
select 
distinct market_name from  silver.daily_pricing_silver
where lakehouse_updated_date > 
(select nvl(max(PROCESSED_TABLE_DATE_TIME), "1900-01-01") 
FROM pricing_analytics.processrunlogs.deltalakehouse_process_runs
WHERE process_name = 'reportingDimensionTablesLoad' AND process_status = 'Completed'
);


In [0]:
create or replace table pricing_analytics.silver.reporting_dim_market_stage_2 AS 
select 
       silverDim.market_name,
       (select nvl(max(MARKET_ID), 0 ) from pricing_analytics.gold.reporting_dim_market_gold)+ row_number() over(order by silverDim.market_name) as MARKET_ID,
       current_timestamp() as lakehouse_inserted_date,
       current_timestamp() as lakehouse_updated_date
       FROM pricing_analytics.silver.reporting_dim_market_stage_1 silverDim  
       LEFT JOIN pricing_analytics.gold.reporting_dim_market_gold goldDim
       on silverDim.market_name = golddim.MARKET_NAME    
       where isnull(golddim.MARKET_NAME)
       ;

In [0]:
insert into pricing_analytics.gold.reporting_dim_market_gold
select * from pricing_analytics.silver.reporting_dim_market_stage_2;

In [0]:
select * from  pricing_analytics.gold.reporting_dim_market_gold ; 

In [0]:
insert into pricing_analytics.processrunlogs.deltalakehouse_process_runs
(PROCESS_NAME,PROCESS_STATUS,PROCESSED_TABLE_DATE_TIME)
select "reportingDimensionTablesLoad","Completed", max(lakehouse_updated_date) from pricing_analytics.silver.daily_pricing_silver 
